<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/develop/gui/dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ─── SETUP FOR DASHBOARD ───

import os, sys

try:
    # Step 1: Clone the GitHub repository if not already present
    if not os.path.exists("/content/Cloud_Computing"):
        !git clone https://github.com/Method-for-Software-System-Development/Cloud_Computing.git /content/Cloud_Computing

    # Step 2: Change directory to project root
    %cd /content/Cloud_Computing

    # Step 3: Checkout the 'develop' branch
    !git fetch origin -q
    !git checkout develop -q

    # Step 4: Add 'logic' directory to Python path
    sys.path.append("/content/Cloud_Computing/logic")

    # Step 5: Install required Python packages (quietly)
    %pip install -q importnb
    %pip install -q paho-mqtt
    %pip install -q -U gradio
    %pip install -q firebase
    %pip install requests beautifulsoup4
    %pip install -q matplotlib

    # Step 6: Import required notebooks from 'logic'
    from importnb import Notebook
    with Notebook():
        import user_controller as uc
        import mqqt_sim_indoor as indoor
        import mqqt_sim_outdoor as outdoor
        import search_index as search
        import sensors_stats as stats
        import Fault_controller as fc
        import repair_controller as rc

    print("✅ Setup completed successfully.")

except Exception as e:
    print("❌ Setup failed:", str(e))

Cloning into '/content/Cloud_Computing'...
remote: Enumerating objects: 596, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 596 (delta 101), reused 37 (delta 37), pack-reused 458 (from 1)
Receiving objects: 100% (596/596), 4.80 MiB | 10.20 MiB/s, done.
Resolving deltas: 100% (348/348), done.
/content/Cloud_Computing
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.0

In [5]:
import gradio as gr
import pytz
from datetime import datetime

# State variable to track login status
is_logged_in = gr.State(False)

with gr.Blocks(css=r"""
:root {
  --dark-blue: #1D3557;
  --blue: #457B9D;
  --light-blue: #A8DADC;
  --mint: #F1FAEE;
  --red: #E63946;
}

#header {
  background: var(--blue);
  padding: 0.8rem 1.2rem;
  display: flex;
  align-items: center;
  min-height: 100px;
  border-radius: 18px;
}

#right_header_panel {
  display: flex;
  flex-direction: column;
  align-items: flex-end;
  justify-content: center;
  height: 100%;
  padding-bottom: 16px;
}

#clock {
    background: var(--blue) !important;
    width: 100% !important;
    text-align: center !important;
    border: 2px solid transparent !important;
    display: flex !important;
    flex-direction: column !important;
    align-items: center !important;
}
#clock textarea {
    font-size: 2rem !important;
    font-weight: bold !important;
    color: white !important;
    background: transparent !important;
    border: none !important;
    outline: none !important;
    text-align: right !important;
    resize: none !important;
    box-shadow: none !important;
    margin: 0 !important;
    padding: 0 !important;
    height: 40px !important;
}

#left_panel {
  background: var(--light-blue);
  padding: 0.8rem 1.2rem;
  display: flex;
  border-radius: 18px;
}

table {
  margin-left: auto;
  margin-right: auto;
  width: 100%;
  background-color: white;
}
th, td {
  background-color: white;
}

#main_panel {
  background: var(--mint);
  padding: 0.8rem 1.2rem;
  display: flex;
  border-radius: 18px;
}

#boxes-row {
  background: var(--mint) !important;
}

.sensor-box {
    border-radius: 18px !important;
    font-weight: bold !important;
    padding: 32px 0px !important;
    width: 210px !important;
    text-align: center !important;
    margin: 12px 18px 8px 18px !important;
    border: 2px solid transparent !important;
    display: flex !important;
    flex-direction: column !important;
    align-items: center !important;
}
.sensor-box textarea {
    font-size: 2.2rem !important;
    font-weight: bold !important;
    color: black !important;
    background: transparent !important;
    border: none !important;
    outline: none !important;
    text-align: center !important;
    resize: none !important;
    box-shadow: none !important;
    margin: 0 !important;
    padding: 0 !important;
    height: 40px !important;
}
.sensor-box label, .sensor-box span {
    font-size: 1.2rem !important;
    color: black !important;
    font-weight: 700 !important;
    text-align: center !important;
    display: block !important;
    margin-bottom: 8px !important;
}
.generating.svelte-ls20lj {
    border: 2px solid transparent !important;
    box-shadow: none !important;
}
div.svelte-633qhp {
    background: transparent !important;
    border: none !important;
}

#indoor-sensor-humidity     { background: #FFADAD !important; }
#indoor-sensor-temperature  { background: #FFD6A5 !important; }
#indoor-sensor-pressure     { background: #FDFFB6 !important; }
#outdoor-sensor-humidity    { background: #CAFFBF !important; }
#outdoor-sensor-temperature { background: #A0C4FF !important; }
#outdoor-sensor-dlight      { background: #BDB2FF !important; }

.fault-low {{
    background-color: #fff176;
    color: white;
    padding: 24px;
    border-radius: 20px;
    font-family: sans-serif;
}}
.fault-medium {{
    background-color: #ffb74d;
    color: white;
    padding: 24px;
    border-radius: 20px;
    font-family: sans-serif;
}}
.fault-high {{
    background-color: #ef5350;
    color: white;
    padding: 24px;
    border-radius: 20px;
    font-family: sans-serif;
}}
#main-fault * {{
    color: black !important;
}}

#repair-box {{
    background-color: white;
    color: black;
    padding: 24px;
    border-radius: 20px;
    font-family: sans-serif;
}}
#repair-box * {{
    color: black !important;
}}
.repair-checkbox {{
  display: flex !important;
  flex-direction: row !important;
  gap: 20px;
  flex-wrap: wrap;
}}

""") as demo:

    # --------------- HEADER ----------------

    with gr.Row(elem_id="header", equal_height=True):
        with gr.Column(scale=4):
            gr.HTML("""
            <div>
              <h1 style="margin:0;font-size:2rem;font-weight:800;color:white;">OptiLine</h1>
              <h3 style="margin:0;font-size:1.2rem;opacity:0.8;color:white;">CIM & Robotics Lab - Braude College of Engineering</h3>
            </div>
            """)
        with gr.Column(scale=1, elem_id="right_header_panel"):
            def get_time():
                """
                Returns the current time in Israel timezone.
                """
                tz = pytz.timezone('Asia/Jerusalem')
                now = datetime.now(tz)
                return now.strftime("%H:%M:%S - %d/%m/%Y")

            clock_txt = gr.Textbox(label="", value=get_time, every=1, interactive=False, elem_id="clock")

    # --------------- MAIN DISPLAY ZONES ----------------

    with gr.Row(equal_height=True):

      with gr.Column(elem_id="left_panel", scale=1):

        # ─── LEFT PANEL: before login ───
        with gr.Column(visible=True) as left_login_panel:
            gr.Markdown("## Sign In:")
            gr.Markdown("### Welcome to the OptiLine system!")
            gr.Markdown("### Please sign in with your username and password.<br> If you don't have an account, contact the lab administrator for access.")
            username = gr.Textbox(label="Username")
            password = gr.Textbox(label="Password", type="password")
            login_msg  = gr.Markdown("")
            with gr.Row():
                clear_btn = gr.Button("Clear")
                submit_btn = gr.Button("Submit")

        # ─── LEFT PANEL: after login ───
        with gr.Column(visible=False) as left_dashboard_panel:
            welcome_txt = gr.Markdown("")
            toGuide = gr.Button("User Guide")
            logout_btn = gr.Button("Logout")
            gr.Markdown("## Main Menu:")
            toSensors = gr.Button("Sensors Dashboard")
            toSearch = gr.Button("MQTT Search Engine")
            toStats = gr.Button("Statistics")
            toFsim = gr.Button("Fault Simulator")
            toDirectory = gr.Button("User Directory")
            gr.Markdown("## Leaderboard:")
            leaderboard_txt = gr.Markdown("")

      with gr.Column(elem_id="main_panel", scale=3):

        # ─── RIGHT PANEL: about/info ───
        with gr.Column(visible=True) as right_info_panel:
            # Load external markdown content
            with open("gui/about.md", "r", encoding="utf-8") as f:
                about_content = f.read()
            gr.Markdown(value=about_content)


        # ─── RIGHT PANEL: live sensors ───
        with gr.Column(visible=False) as right_sensor_panel:
            gr.Markdown("# Live Indoor Sensors Data:")
            with gr.Row(elem_id="boxes-row"):
                humidity_box_in = gr.Textbox(label="Indoor Humidity", show_label=True, elem_id="indoor-sensor-humidity", elem_classes="sensor-box", interactive=False)
                temp_box_in     = gr.Textbox(label="Indoor Temperature", show_label=True, elem_id="indoor-sensor-temperature", elem_classes="sensor-box", interactive=False)
                pressure_box_in = gr.Textbox(label="Pressure", show_label=True, elem_id="indoor-sensor-pressure", elem_classes="sensor-box", interactive=False)

            gr.Markdown("# Live Outdoor Sensors Data:")
            with gr.Row(elem_id="boxes-row"):
                humidity_box_out = gr.Textbox(label="Outdoor Humidity", show_label=True, elem_id="outdoor-sensor-humidity", elem_classes="sensor-box", interactive=False)
                temp_box_out     = gr.Textbox(label="Outdoor Temperature", show_label=True, elem_id="outdoor-sensor-temperature", elem_classes="sensor-box", interactive=False)
                dlight_box_out   = gr.Textbox(label="DayLight (Illuminance)", show_label=True, elem_id="outdoor-sensor-dlight", elem_classes="sensor-box", interactive=False)

        def card_streamer():
            """
            Generator function that yields live sensor data or simulated data for the dashboard.

            Args:
                mode (str): Data source mode.
                    - "mqtt": Receives live data from physical sensors via MQTT.
                    - "simulation": Receives data from a simulated data stream.
                    Default is "mqtt".

            Yields:
                dict: A dictionary mapping each dashboard textbox to its current sensor value.
            """
            # Get data streams for indoor and outdoor sensors according to the selected mode
            for indoor_data, outdoor_data in zip(indoor.get_live_data_stream(mode="simulation"),
                                                outdoor.get_live_data_stream(mode="simulation")):
                yield {
                    humidity_box_in:  f"{indoor_data.get('Humidity', 'N/A')} %",
                    temp_box_in:      f"{indoor_data.get('Temperature', 'N/A')} °C",
                    pressure_box_in:  f"{indoor_data.get('Pressure', 'N/A')} hPa",
                    humidity_box_out: f"{outdoor_data.get('Humidity', 'N/A')} %",
                    temp_box_out:     f"{outdoor_data.get('Temperature', 'N/A')} °C",
                    dlight_box_out:   f"{outdoor_data.get('Dlight', 'N/A')} Lux"
                }

        demo.load(
            fn=card_streamer,
            outputs=[
                humidity_box_in, temp_box_in, pressure_box_in,
                humidity_box_out, temp_box_out, dlight_box_out
            ]
        )

        # ─── RIGHT PANEL: guide ───
        with gr.Column(visible=False) as guide_panel:
            # Load external markdown content
            with open("gui/user_guide.md", "r", encoding="utf-8") as f:
                user_guide_content = f.read()
            gr.Markdown(value=user_guide_content)


        # ─── RIGHT PANEL: search ───
        with gr.Column(visible=False) as search_panel:
            gr.Markdown("# 🔍 MQTT.org Website Search Engine")
            gr.Markdown("Type keywords to search the knowledge base. Results will appear below with clickable links.")
            search_query = gr.Textbox(label="Enter words to search", placeholder="Type your query here...")
            search_btn = gr.Button("Search")
            search_results = gr.HTML(label="Search Results")

        # ─── RIGHT PANEL: statistics ───
        with gr.Column(visible=False) as stats_panel:
            gr.Markdown("# 📊 Sensors Data Statistics")
            gr.Markdown(
                """
                View and analyze historical data collected every minute from indoor and outdoor sensors.
                Select a date and hour to explore trends in temperature, humidity, light, and pressure over time.
                Interactive plots help you compare sensor values and spot anomalies.
                """
            )

            # Retrieve date strings and mapping from statistics module functions
            unique_date_strings = stats.get_unique_date_strings()
            date_to_hours_map = stats.get_date_to_hours_map()

            if unique_date_strings:
                # Initialize dropdown values
                initial_selected_date_str = unique_date_strings[0]
                initial_hour_choices = date_to_hours_map.get(initial_selected_date_str, [])
                initial_selected_hour_str = initial_hour_choices[0] if initial_hour_choices else None

                date_dropdown = gr.Dropdown(
                    choices=unique_date_strings,
                    value=initial_selected_date_str,
                    label="Select Date"
                )

                time_dropdown = gr.Dropdown(
                    choices=initial_hour_choices,
                    value=initial_selected_hour_str,
                    label="Select Start Hour for 1 Hour Interval"
                )

                with gr.Row():
                    plot_dlight_out = gr.Plot(label="Outdoor Dlight")
                    plot_temp_out = gr.Plot(label="Outdoor Temperature")
                with gr.Row():
                    plot_humidity_out = gr.Plot(label="Outdoor Humidity")
                    plot_humidity_in = gr.Plot(label="Indoor Humidity")
                with gr.Row():
                    plot_temp_in = gr.Plot(label="Indoor Temperature")
                    plot_pressure_in = gr.Plot(label="Indoor Pressure")
                with gr.Row():
                    plot_distance_in = gr.Plot(label="Indoor Distance")

                plot_outputs_list = [
                    plot_dlight_out, plot_temp_out, plot_humidity_out,
                    plot_humidity_in, plot_temp_in, plot_pressure_in, plot_distance_in
                ]

                # Update hour dropdown and plots when the selected date changes
                def update_hour_dropdown_and_plots(selected_date_str_event):
                    # Always fetch the latest mapping to ensure up-to-date options
                    hours_map = stats.get_date_to_hours_map()
                    new_hour_choices = hours_map.get(selected_date_str_event, [])
                    new_selected_hour = new_hour_choices[0] if new_hour_choices else None

                    plot_updates = stats.generate_plots(selected_date_str_event, new_selected_hour)
                    return [gr.update(choices=new_hour_choices, value=new_selected_hour)] + list(plot_updates if plot_updates else [None]*7)

                date_dropdown.change(
                    fn=update_hour_dropdown_and_plots,
                    inputs=date_dropdown,
                    outputs=[time_dropdown] + plot_outputs_list
                )

                time_dropdown.change(
                    fn=stats.generate_plots,
                    inputs=[date_dropdown, time_dropdown],
                    outputs=plot_outputs_list
                )

                # Initial plot generation on UI load
                demo.load(
                    fn=stats.generate_plots,
                    inputs=[date_dropdown, time_dropdown],
                    outputs=plot_outputs_list
                )
            else:
                gr.Markdown("No data available to determine dates from Firebase.")

        # ─── RIGHT PANEL: fault simulator ───
        with gr.Column(visible=False) as fsim_panel:

              # ---------- Active faults Helpers ----------

              def format_datetime(iso_str):
                  dt = datetime.fromisoformat(iso_str)
                  return dt.strftime("%-d/%-m, %-I:%M %p")

              def get_main_fault():
                  faults = fc.fb.get_active_faults()
                  if not isinstance(faults, dict) or not faults:
                      return None, "", "", ""

                  first_key = next(iter(faults))  # This is the actual Firebase key
                  fault = faults[first_key]

                  title_words = fault.get("title", "").split()
                  sensor1 = " ".join(title_words[:2])
                  sensor2 = " ".join(title_words[2:]) if len(title_words) > 2 else ""

                  return fault, sensor1, sensor2, first_key

              def get_sort_key(fault, sort_mode, sensor_order):
                  severity_rank = {"Low": 1, "Medium": 2, "High": 3}
                  sev = severity_rank.get(fault.get("severity", ""), 0)

                  if sensor_order == "Indoor → Outdoor":
                      sensor_val = 0 if "Indoor" in fault.get("sensor", "") or "Indoor" in fault.get("title", "") else 1
                  elif sensor_order == "Outdoor → Indoor":
                      sensor_val = 1 if "Indoor" in fault.get("sensor", "") or "Indoor" in fault.get("title", "") else 0
                  else:
                      sensor_val = 0

                  if sort_mode == "Low → High":
                      return (sev, sensor_val)
                  elif sort_mode == "High → Low":
                      return (-sev, sensor_val)
                  else:  # By Time
                      return (sensor_val,) if sensor_order != "Original Order" else ()

              def active_faults_table_data(sort_mode="By Time", sensor_order="Original Order", main_fault_title=None):
                  faults = fc.fb.get_active_faults()
                  if not isinstance(faults, dict) or not faults:
                      return None, []

                  fault_items = list(faults.items())

                  # Exclude the current main fault
                  if main_fault_title:
                      fault_items = [item for item in fault_items if item[0] != main_fault_title]

                  if sort_mode != "By Time" or sensor_order != "Original Order":
                      fault_items = sorted(
                          fault_items,
                          key=lambda x: get_sort_key(x[1], sort_mode, sensor_order)
                      )

                  rows = []  # Will contain the final table rows (for UI display)
                  keys = []  # Will store the corresponding fault keys

                  for key, fault in fault_items:
                      iso_ts = fault.get("timestamp", "")
                      try:
                          dt = datetime.fromisoformat(iso_ts)
                          time_str = dt.strftime("%-d/%-m, %-I:%M%p").upper()
                      except Exception:
                          time_str = iso_ts[:16].replace("T", " ")

                      # Append the fault key and display row to their respective lists
                      keys.append(key)
                      rows.append([
                          time_str,
                          fault.get("title", ""),
                          fault.get("severity", ""),
                          fault.get("status", "")
                      ])

                  # Return the list of fault keys and their corresponding table rows
                  return keys, rows

              def on_table_click(evt: gr.SelectData, keys):
                  row_index = evt.index[0]
                  main_key = keys[row_index]
                  faults = fc.fb.get_active_faults()
                  fault = faults.get(main_key, {})
                  severity = fault.get("severity", "")
                  severity_class = {
                      "Low": "fault-low",
                      "Medium": "fault-medium",
                      "High": "fault-high"
                  }.get(severity, "fault-low")

                  return (
                      main_key,
                      *update_repair_box(main_key),
                      *update_fault_info(main_key),
                      gr.update(elem_classes=[severity_class])
                  )

              def update_repair_box(main_key):
                  # Get all currently active faults from Firebase (or fault controller)
                  faults = fc.fb.get_active_faults()
                  # Find the specific fault by its key, if faults exist
                  fault = faults.get(main_key) if faults else None

                  # If no fault found, return updates that disable or clear the repair UI elements
                  if not fault:
                      return (
                          gr.update(value="No Active Faults To Repair"),  # XP display text
                          gr.update(value=""),                            # Sensor name text cleared
                          gr.update(visible=False),                       # Hide start button
                          gr.update(choices=[], value=[], interactive=False),  # Clear and disable checklist
                          gr.update(visible=False),                       # Hide finish button
                      )

                  # Map fault severity to XP reward values
                  severity_to_xp = {"Low": 50, "Medium": 100, "High": 200}
                  # Get XP based on severity, default to 50 if unknown severity
                  xp = severity_to_xp.get(fault.get("severity", "Low"), 50)

                  title = fault.get("title", "")
                  title_words = title.split()
                  # Extract first two words from title for sensor snippet display
                  sensor_snippet = " ".join(title_words[:2]) if len(title_words) >= 2 else title

                  # Return UI updates to show XP, sensor info, enable start button, checklist with actions,
                  # and keep finish button hidden initially
                  return (
                      gr.update(value=f"Earn {xp} XP by fixing"),                             # XP reward text
                      gr.update(value=f"{sensor_snippet} Sensor Whithin 10 Minutes"),        # Sensor name text
                      gr.update(visible=True),                                                # Show start button
                      gr.update(choices=fault["actions"], value=[], interactive=False),       # Checklist with fault actions, no selection yet
                      gr.update(visible=False),                                               # Hide finish button initially
                  )

              def update_fault_info(main_key):
                  # Get current active faults
                  faults = fc.fb.get_active_faults()
                  # Get specific fault by key
                  fault = faults.get(main_key) if faults else None

                  # If fault not found, return placeholders indicating no data
                  if not fault:
                      return (
                          gr.update(value="--"),       # fault time placeholder
                          gr.update(value="--"),       # fault location placeholder
                          gr.update(value="No actions available.")  # fault actions placeholder text
                      )

                  # Parse ISO timestamp and format it to readable string (e.g., 5/22, 3:45PM)
                  iso_ts = fault.get("timestamp", "")
                  try:
                      dt = datetime.fromisoformat(iso_ts)
                      time_str = dt.strftime("%-d/%-m, %-I:%M%p").upper()
                  except Exception:
                      # Fallback formatting if parsing fails
                      time_str = iso_ts[:16].replace("T", " ")

                  title = fault.get("title", "")
                  actions = fault.get("actions", [])

                  # Format list of actions as bullet points for display
                  action_lines = "\n".join(f"- {a}" for a in actions)

                  # Return UI updates for fault info display elements
                  return (
                      gr.update(value=f"**{time_str}**"),   # Formatted fault time
                      gr.update(value=f"### {title}"),      # Fault title
                      gr.update(value=action_lines)          # List of fault actions
                  )

              # ----------- Resolved faults Helper -----------
              def resolved_faults_table_data():
                  try:
                      faults = fc.fb.get_resolved_faults()
                      if not isinstance(faults, dict) or not faults:
                          return []

                      rows = []
                      for _, fault in faults.items():
                          try:
                              time_str = format_datetime(fault.get("timestamp", ""))
                          except:
                              time_str = fault.get("timestamp", "")[:16].replace("T", " ")

                          rows.append([
                              time_str,
                              fault.get("sensor", ""),
                              fault.get("severity", ""),
                              fault.get("repaired_by", ""),
                              fault.get("status", "Resolved")
                          ])
                      return rows
                  except Exception as e:
                      print("❌ resolved_faults_table_data() error:", e)
                      return []

              # ---------- Extracted ----------
              presented_fault, sensor1, sensor2, presented_key = get_main_fault()
              time_str = format_datetime(presented_fault["timestamp"])
              xp_value = {"Low": 50, "Medium": 100, "High": 200}.get(presented_fault["severity"], 50)

              # -------- GUI: First Row ----------
              with gr.Row():
                main_fault_column = gr.Column(elem_id="main-fault", elem_classes=[severity_class])
                with main_fault_column:
                      gr.Markdown("## FAULT DETECTED")
                      fault_time_text = gr.Markdown("")
                      fault_sensor_name = gr.Markdown("")
                      gr.Markdown("#### Suggested Actions:")
                      fault_actions_box = gr.Markdown("")

                with gr.Column(elem_id="repair-box", scale=1):
                      gr.Markdown("## REPAIR CHALLENGE")
                      repair_xp_text = gr.Markdown("")
                      repair_sensor_text = gr.Markdown("")
                      gr.Markdown("#### Repair Steps")

                      repair_checklist = gr.CheckboxGroup(
                          choices=[],
                          value=[],
                          interactive=False,
                          label="",
                          elem_classes=["repair-checklist"]
                      )

                      start_btn = gr.Button("Start Repair", visible=True)
                      finish_btn = gr.Button("Finish Repair", visible=False, interactive=False)
                      status_output = gr.Textbox(label="System Message", interactive=False)

              # -------- Second Row ----------
              with gr.Row():
                  with gr.Column():
                      with gr.Row():
                          gr.Markdown("### Active Faults")

                          sensor_filter = gr.Dropdown(
                              choices=["Original Order", "Indoor → Outdoor", "Outdoor → Indoor"],
                              value="Original Order",  # default
                              label="",
                              interactive=True,
                              scale=1,
                              elem_classes=["small-dropdown"]
                          )
                          sort_choice = gr.Dropdown(
                              choices=["By Time", "Low → High", "High → Low"],
                              value="By Time",      # default
                              label="",
                              interactive=True,
                              scale=1,
                              elem_classes=["small-dropdown"]
                          )


                      table_headers = gr.Dataframe(
                          headers=["Time", "Sensor", "Severity", "Status"],
                          value=[],
                          row_count=0,
                          col_count=(4, "fixed"),
                          interactive=True,
                          label=""
                  )
              # -------- Third Row ----------
              with gr.Row():
                  with gr.Column():
                      gr.Markdown("### Resolved Faults")
                      resolved_table = gr.Dataframe(
                          headers=["Time Detected", "Sensor", "Severity", "Repaired by", "Status"],
                          value=[],                     # Start empty
                          row_count=0,
                          col_count=(5, "fixed"),
                          interactive=False,
                          label=""
                      )

              # ---------- Logic ----------
              def start_real_repair(main_key):
                  faults = fc.fb.get_active_faults()                                      # Get the active fault dictionary from Firebase
                  fault = faults.get(main_key)                                            # Get the specific fault by key

                  if not fault or not isinstance(fault, dict):
                      return (
                          gr.update(choices=[], value=[], interactive=False),             # checklist
                          gr.update(visible=True),                                        # start_btn
                          gr.update(visible=False)                                        # finish_btn
                      )

                  # Make sure actions are a list
                  actions = fault.get("actions")
                  # Start the repair (write to Firebase)
                  msg = rc.start_repair(main_key)

                  return (
                      gr.update(choices=actions, value=[], interactive=True),             # checklist
                      gr.update(visible=False),                                           # start_btn
                      gr.update(visible=True),                                            # finish_btn
                      msg                                                                 # status_output
                  )

              def handle_checkbox_update(selected, main_key):
                  faults = fc.fb.get_active_faults()
                  fault = faults[main_key]
                  expected = fault.get("actions", [])
                  return gr.update(interactive=(selected == expected))


              def complete_real_repair(main_key):
                  msg = rc.complete_repair(main_key)  # Again, use the Firebase key
                  return (
                      gr.update(visible=False),
                      gr.update(value="Start Repair", visible=True, interactive=True),
                      msg
                  )


              def load_active_faults():
                  rows = active_faults_table_data()
                  return gr.update(value=rows)


              # ---------- Bindings ----------
              presented_key_state = gr.State(presented_key)
              fault_keys_state = gr.State([])

              start_btn.click(
                  fn=start_real_repair,
                  inputs=[presented_key_state],
                  outputs=[repair_checklist, start_btn, finish_btn, status_output]
              )

              finish_btn.click(
                  fn=complete_real_repair,
                  inputs=[presented_key_state],
                  outputs=[finish_btn, start_btn, status_output]
              )

              repair_checklist.change(
                  fn=handle_checkbox_update,
                  inputs=[repair_checklist, presented_key_state],
                  outputs=finish_btn
              )

              sort_choice.change(
                  fn=lambda sort, sensor, key: active_faults_table_data(sort, sensor, key)[1],
                  inputs=[sort_choice, sensor_filter, presented_key_state],
                  outputs=[table_headers]
              )

              sensor_filter.change(
                  fn=lambda sort, sensor, key: active_faults_table_data(sort, sensor, key)[1],
                  inputs=[sort_choice, sensor_filter, presented_key_state],
                  outputs=[table_headers]
              )

              # Register a selection event on the fault table
              table_headers.select(
              # Function to call when a row is selected
              fn=on_table_click,
              inputs=[fault_keys_state],
              outputs=[
                  presented_key_state,   # Update the state of the currently presented fault key
                  repair_xp_text,        # Update the XP (score) text area for the selected fault
                  repair_sensor_text,    # Update the sensor name or location text for the selected fault
                  start_btn,
                  repair_checklist,      # Load and show the checklist steps (if any) for the fault
                  finish_btn,
                  fault_time_text,
                  fault_sensor_name,
                  fault_actions_box,
                  main_fault_column,
                  ]
              )


              demo.load(
              fn=lambda key: (
                  *update_repair_box(key),
                  active_faults_table_data("By Time", "Original Order", key)[1],  # rows
                  active_faults_table_data("By Time", "Original Order", key)[0],  # keys
                  resolved_faults_table_data(),
                  *update_fault_info(key)
              ),
              inputs=[presented_key_state],
              outputs=[
                  repair_xp_text,
                  repair_sensor_text,
                  start_btn,
                  repair_checklist,
                  finish_btn,
                  table_headers,
                  fault_keys_state,
                  resolved_table,
                  fault_time_text,         # UI elements at top
                  fault_sensor_name,
                  fault_actions_box
              ]
          )
        # ─── RIGHT PANEL: directory ───
        with gr.Column(visible=False) as directory_panel:
            gr.Markdown("# 👥 User Directory")
            gr.Markdown("Browse the list of authorized system users and view their contact information.")
            phonebook_txt = gr.Markdown("")

    # --------------- LOGIC ----------------

    def clear_fields():
        """
        Clears the username and password input fields.

        Returns:
            tuple: Empty strings for username and password.
        """
        return "", ""

    def do_login(user, pw):
        """
        Handles user login. Updates all dashboard panels and user info on successful authentication.

        Args:
            user (str): Username entered in the login form.
            pw (str): Password entered in the login form.

        Returns:
            list: Gradio UI updates for all relevant components.
        """
        if uc.login(user, pw):
            # Retrieve user's profile from Firebase
            _, profile = uc.fb.get_user(user)
            name = profile.get("first_name", user)
            role = profile.get("role", "")
            user_name = profile.get("username", user)

            # Build a personalized welcome message
            welcome_text = f"### Welcome back, **{name}** ({role})"

            # Retrieve leaderboard and user's rank
            leaderboard_list, user_rank = uc.get_leaderboard(user_name)

            # Build leaderboard Markdown table with medals
            leaderboard_md = "#### \n| Rank | Username | Score |\n|:----:|:-----------|:------:|\n"
            for rank, uname, score in leaderboard_list:
                medal = ""
                if rank == 1: medal = "🥇"
                elif rank == 2: medal = "🥈"
                elif rank == 3: medal = "🥉"
                you = " <b>(You)</b>" if uname == user_name else ""
                name_str = f"**{uname}{you}**" if uname == user_name else uname
                score_str = f"**{score}**" if uname == user_name else str(score)
                leaderboard_md += f"| {rank} {medal} | {name_str} | {score_str} |\n"

            # Show dashboard panels after login: left_dashboard_panel, right_sensor_panel
            # Hide left_login_panel, right_info_panel, search_panel, stats_panel
            return [
                gr.update(visible=False),            # left_login_panel
                gr.update(visible=True),             # left_dashboard_panel
                gr.update(visible=False),            # right_info_panel
                gr.update(visible=True),             # right_sensor_panel
                gr.update(value=True),               # is_logged_in
                "",                                  # login_msg
                gr.update(value=welcome_text),       # welcome_txt
                gr.update(value=leaderboard_md),     # leaderboard_txt
            ]
        else:
            # Failed login: show error message, keep panels unchanged
            return [
                gr.update(), gr.update(), gr.update(), gr.update(),
                gr.update(value=False), "Incorrect username or password",
                gr.update(), gr.update()
            ]

    def do_logout():
        """
        Handles user logout. Restores the login screen and hides all dashboard panels.

        Returns:
            list: Gradio UI updates for all relevant components.
        """
        return [
            gr.update(visible=True),    # left_login_panel
            gr.update(visible=False),   # left_dashboard_panel
            gr.update(visible=True),    # right_info_panel
            gr.update(visible=False),   # right_sensor_panel
            gr.update(visible=False),   # search_panel
            gr.update(visible=False),   # stats_panel
            gr.update(visible=False),   # guide_panel
            gr.update(visible=False),   # fsim_panel
            gr.update(visible=False),   # directory_panel
            gr.update(value=False),     # is_logged_in
            "",                         # login_msg
            gr.update(value=""),        # welcome_txt
            gr.update(value=""),        # leaderboard_txt
        ]

    def show_sensor_panel():
        """
        Shows the live sensors panel, hides all other panels.
        """
        return [
            gr.update(visible=True),   # right_sensor_panel
            gr.update(visible=False),  # search_panel
            gr.update(visible=False),  # stats_panel
            gr.update(visible=False),  # guide_panel
            gr.update(visible=False),  # fsim_panel
            gr.update(visible=False),  # directory_panel
        ]

    def show_search_panel():
        """
        Shows the search panel, hides all other panels.
        """
        return [
            gr.update(visible=False),  # right_sensor_panel
            gr.update(visible=True),   # search_panel
            gr.update(visible=False),  # stats_panel
            gr.update(visible=False),  # guide_panel
            gr.update(visible=False),  # fsim_panel
            gr.update(visible=False),  # directory_panel
        ]

    def show_stats_panel():
        """
        Shows the statistics panel, hides all other panels.
        """
        return [
            gr.update(visible=False),  # right_sensor_panel
            gr.update(visible=False),  # search_panel
            gr.update(visible=True),   # stats_panel
            gr.update(visible=False),  # guide_panel
            gr.update(visible=False),  # fsim_panel
            gr.update(visible=False),  # directory_panel
        ]

    def show_guide_panel():
        """
        Shows the usage guide panel, hides all other panels.
        """
        return [
            gr.update(visible=False),  # right_sensor_panel
            gr.update(visible=False),  # search_panel
            gr.update(visible=False),  # stats_panel
            gr.update(visible=True),   # guide_panel
            gr.update(visible=False),  # fsim_panel
            gr.update(visible=False),  # directory_panel
        ]

    def show_fsim_panel():
        """
        Shows the fault simulator panel, hides all other panels.
        """
        return [
            gr.update(visible=False),  # right_sensor_panel
            gr.update(visible=False),  # search_panel
            gr.update(visible=False),  # stats_panel
            gr.update(visible=False),  # guide_panel
            gr.update(visible=True),  # fsim_panel
            gr.update(visible=False),   # directory_panel
        ]

    def show_directory_panel():
        """
        Shows the user directory panel, hides all other panels.
        """
        return [
            gr.update(visible=False),  # right_sensor_panel
            gr.update(visible=False),  # search_panel
            gr.update(visible=False),  # stats_panel
            gr.update(visible=False),  # guide_panel
            gr.update(visible=False),  # fsim_panel
            gr.update(visible=True),   # directory_panel
        ]

    def generate_phonebook_table():
        """
        Generates a Markdown table representing the user directory (phonebook),
        including name, role, and phone number, sorted alphabetically by last name.

        Returns:
            str: A Markdown-formatted string containing the directory table.
        """
        status, phonebook = uc.fb.get_phonebook()

        if status != 200 or not phonebook:
            return "Failed to load phonebook."

        # Sort alphabetically by last name
        phonebook.sort(key=lambda x: x["name"].split()[-1].lower())

        # Build Markdown table: Name (left), Role (middle), Phone (right)
        md = "#### \n| Name | Role | Phone |\n|:----------------|:----------------------|:------------|\n"
        for entry in phonebook:
            name = entry.get("name", "N/A")
            role = entry.get("role", "N/A")
            phone = entry.get("phone", "N/A")
            md += f"| {name} | {role} | {phone} |\n"

        return md

    # Button callbacks:

    toSensors.click(fn=show_sensor_panel, outputs=[right_sensor_panel, search_panel, stats_panel, guide_panel, fsim_panel, directory_panel])
    toSearch.click(fn=show_search_panel, outputs=[right_sensor_panel, search_panel, stats_panel, guide_panel, fsim_panel, directory_panel])
    toStats.click(fn=show_stats_panel, outputs=[right_sensor_panel, search_panel, stats_panel, guide_panel, fsim_panel, directory_panel])
    toGuide.click(fn=show_guide_panel, outputs=[right_sensor_panel, search_panel, stats_panel, guide_panel, fsim_panel, directory_panel])
    toFsim.click(fn=show_fsim_panel, outputs=[right_sensor_panel, search_panel, stats_panel, guide_panel, fsim_panel, directory_panel])

    toDirectory.click(
        fn=lambda: [*show_directory_panel(), generate_phonebook_table()],
        outputs=[right_sensor_panel, search_panel, stats_panel, guide_panel, directory_panel, phonebook_txt]
    )

    clear_btn.click(fn=clear_fields, outputs=[username, password])

    submit_btn.click(
        fn=do_login,
        inputs=[username, password],
        outputs=[
            left_login_panel,
            left_dashboard_panel,
            right_info_panel,
            right_sensor_panel,
            is_logged_in,
            login_msg,
            welcome_txt,
            leaderboard_txt,
        ]
    )

    logout_btn.click(
        fn=do_logout,
        outputs=[
            left_login_panel,
            left_dashboard_panel,
            right_info_panel,
            right_sensor_panel,
            search_panel,
            stats_panel,
            guide_panel,
            fsim_panel,
            directory_panel,
            is_logged_in,
            login_msg,
            welcome_txt,
            leaderboard_txt,
        ]
    )

    search_btn.click(
        fn=search.search_words,
        inputs=search_query,
        outputs=search_results
    )

# Launch app
demo.launch()

NameError: name 'severity_class' is not defined